In [1]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import folium
from folium.plugins import HeatMap
from shapely.geometry import MultiPoint

In [3]:
import os
import sys

from os.path import dirname
UTILS_PATH=os.environ.get('REPO') + "/notebooks/commons"
sys.path.append(dirname(UTILS_PATH))

from commons import check_args, gen_paths, coordinates_bounds

In [4]:
import math

def safe_pow(e):
    result = math.inf
    try:
        result = float(2**e)
        return result
    except OverflowError as error:
        return result
    

In [5]:
def load_posts(input_path):
    df = pd.read_parquet(input_path)
    return df

In [6]:
if __name__ == "__main__":
    city = "manizales"
    property_type = "apartamentos"
    post_type = "arriendo"
    
    #city, property_type, post_type = check_args()
    base_path = os.environ.get('REPO')
    
    raw_path, clean_path = gen_paths(city, property_type, post_type)
    
    raw_path = base_path + raw_path
    clean_path = base_path + clean_path

In [7]:
MSG = """
Hay {} publicaciones de {} en total
"""

posts = load_posts(clean_path)
print(MSG.format(posts.shape[0], post_type))
posts.head()


Hay 1027 publicaciones de arriendo en total



,surface,rooms,baths,garages,price,latitude,description,longitude,location,url,admon,estrato,antiguedad,fid,city,property_type,post_type,price_m2
0,65.0,2.0,1.0,0.0,0.65,5.058869,Arrienda apartamento en el barrio La Sultana s...,-75.475807,Manizales La Sultana,https://www.fincaraiz.com.co/apartamento-en-ar...,0.0,4.0,desconocido,5469476,manizales,apartamentos,arriendo,0.010000
1,55.0,2.0,1.0,0.0,0.80,5.058840,Apartamento con excelente iluminación muy bien...,-75.485168,Manizales La Rambla,https://www.fincaraiz.com.co/apartamento-en-ar...,0.0,5.0,1 a 8,5357660,manizales,apartamentos,arriendo,0.014545
2,80.0,2.0,2.0,1.0,1.90,5.052409,Administra Bienes Raíces ofrece para arrendami...,-75.485085,Manizales Camelia,https://www.fincaraiz.com.co/apartamento-en-ar...,0.0,6.0,1 a 8,5670358,manizales,apartamentos,arriendo,0.023750
4,100.0,4.0,2.0,0.0,1.25,5.053482,Espacioso apartamento en un tercer piso con gr...,-75.495567,Manizales Fátima,https://www.fincaraiz.com.co/apartamento-en-ar...,0.0,3.0,1 a 8,5468657,manizales,apartamentos,arriendo,0.012500
5,41.0,3.0,1.0,0.0,0.50,5.070275,"Area de 41 mtrs, excelente ubicacion, muy ilum...",-75.513817,Manizales SANTOS,https://www.fincaraiz.com.co/apartamento-en-ar...,0.0,3.0,16 a 30,5174681,manizales,apartamentos,arriendo,0.012195


## Mapa de calor precios

In [8]:
posts["price"].describe()

count    1027.000000
mean        1.665343
std        14.077540
min         0.075000
25%         0.807000
50%         1.100000
75%         1.500000
max       452.000000
Name: price, dtype: float64

In [9]:
folium_hmap = folium.Map(location=[
    np.mean([coordinates_bounds[city]["lat"]["lower"], coordinates_bounds[city]["lat"]["upper"]]),
    np.mean([coordinates_bounds[city]["lon"]["lower"], coordinates_bounds[city]["lon"]["upper"]])
], zoom_start=13, tiles="OpenStreetMap")

posts.loc[:, "price_exploded"] =  posts["price"].apply(safe_pow)

max_amount = posts['price_exploded'].max()

hm_wide = HeatMap( 
    posts[["latitude", "longitude", "price_exploded"]],
    min_opacity=0.2,
    max_val=max_amount,
    radius=7, blur=1, 
    max_zoom=15, 
    gradient={.1:'yellow', .5: 'orange',  1: 'red'}
)

folium_hmap.add_child(hm_wide)

## Mapa de calor número de publicaciones

In [10]:
from sklearn.cluster import DBSCAN

In [11]:
coords = posts[['latitude','longitude']].values

kms_per_radian = 6371.0088
epsilon = 0.1 / kms_per_radian

min_samples = 20

In [12]:
## Geo clustering with examples
# https://stackoverflow.com/questions/24762435/clustering-geo-location-coordinates-lat-long-pairs-using-kmeans-algorithm-with
# https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

In [13]:
centroids = []
clusters_size = len(db.labels_)

for n in range(clusters_size):
    cluster = posts[['latitude','longitude']][db.labels_ == n].copy()
    if len(cluster) > 0:
        cluster.loc[:, "cluster_id"] = n
        
        multi_point = MultiPoint(cluster.values)
        cluster.loc[:, "cluster_latitude"] = multi_point.centroid.x
        cluster.loc[:, "cluster_longitude"] = multi_point.centroid.y
        
    centroids.append(cluster)
        
clusters_df = pd.concat(centroids)
clusters_df = clusters_df.reset_index(drop=True)
clusters_df.head()

,latitude,longitude,cluster_id,cluster_latitude,cluster_longitude
0,5.06465,-75.507553,0.0,5.06465,-75.507527
1,5.06465,-75.507553,0.0,5.06465,-75.507527
2,5.06465,-75.507553,0.0,5.06465,-75.507527
3,5.06465,-75.507553,0.0,5.06465,-75.507527
4,5.06465,-75.507553,0.0,5.06465,-75.507527


In [14]:
clusters_coordinates = clusters_df[["cluster_id", "latitude", "longitude"]].groupby(["latitude", "longitude"]).count()

coordinates = list(zip(*clusters_coordinates.index))
clusters_coordinates.loc[:, "latitude"] = coordinates[0]
clusters_coordinates.loc[:, "longitude"] = coordinates[1]

clusters_coordinates.loc[:, "count"] = clusters_coordinates["cluster_id"].apply(safe_pow)
clusters_coordinates = clusters_coordinates.reset_index(drop=True)

clusters_coordinates.head()

,cluster_id,latitude,longitude,count
0,4,5.041388,-75.478073,16.0
1,1,5.041420,-75.478203,2.0
2,1,5.041699,-75.478821,2.0
3,1,5.041966,-75.477982,2.0
4,1,5.042030,-75.477509,2.0


In [15]:
folium_hmap = folium.Map(location=[
    np.mean([coordinates_bounds[city]["lat"]["lower"], coordinates_bounds[city]["lat"]["upper"]]),
    np.mean([coordinates_bounds[city]["lon"]["lower"], coordinates_bounds[city]["lon"]["upper"]])
], zoom_start=13, tiles="OpenStreetMap")

max_amount = clusters_coordinates['count'].max()

hm_wide = HeatMap( 
    clusters_coordinates[["latitude", "longitude", "count"]],
    min_opacity=0.2,
    max_val=max_amount,
    radius=7, blur=1, 
    max_zoom=15, 
    gradient={.1:'yellow', .5: 'orange',  1: 'red'}
)

folium_hmap.add_child(hm_wide)